<a href="https://colab.research.google.com/github/adilabduakhanov/Developing_Neuro-Employees_on_GPT/blob/main/%D0%A0%D0%B0%D1%81%D1%88%D0%B8%D1%80%D0%B5%D0%BD%D0%BD%D0%BE%D0%B5_%D1%81%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_Tg_%D0%B1%D0%BE%D1%82%D0%BE%D0%B2_%D0%BD%D0%B0_Aiogram_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q aiogram openai nest_asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 714.0/714.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
import asyncio
import logging
import nest_asyncio
from aiogram import Bot, Dispatcher, types
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, InlineKeyboardMarkup, InlineKeyboardButton
from aiogram.fsm.state import State, StatesGroup
from aiogram.fsm.context import FSMContext
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.filters import Command
from openai import OpenAI

nest_asyncio.apply()

logging.basicConfig(level=logging.INFO)


In [ ]:
from google.colab import userdata
from aiogram import Bot, Dispatcher
from aiogram.fsm.storage.memory import MemoryStorage
from openai import OpenAI

# Получаем токены из секретов Colab
TELEGRAM_TOKEN = userdata.get("TELEGRAM_TOKEN")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

# Проверка (на случай если секрет не добавлен)
if TELEGRAM_TOKEN is None:
    raise ValueError("TELEGRAM_TOKEN не найден в Secrets")

if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY не найден в Secrets")

# Инициализация бота
bot = Bot(token=TELEGRAM_TOKEN)
dp = Dispatcher(storage=MemoryStorage())

# Инициализация OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

print("Токены успешно загружены ✅")


Токены успешно загружены ✅


In [ ]:
tasks = []
deals = []


In [ ]:
class TaskState(StatesGroup):
    waiting_for_name = State()
    waiting_for_time = State()

class DealState(StatesGroup):
    waiting_for_name = State()
    waiting_for_amount = State()
    waiting_for_status = State()

class DealStatusState(StatesGroup):
    waiting_for_new_status = State()


In [ ]:
main_keyboard = ReplyKeyboardMarkup(
    keyboard=[
        [KeyboardButton(text="Добавить задачу")],
        [KeyboardButton(text="Добавить сделку")],
        [KeyboardButton(text="Просмотреть задачи")],
        [KeyboardButton(text="Просмотреть сделки")],
        [KeyboardButton(text="Получить мотивацию")]
    ],
    resize_keyboard=True
)


In [ ]:
@dp.message(Command("start"))
async def start_handler(message: types.Message):
    await message.answer("Привет! Я твой помощник менеджера 🚀", reply_markup=main_keyboard)


In [ ]:
@dp.message(lambda message: message.text == "Добавить задачу")
async def add_task(message: types.Message, state: FSMContext):
    await message.answer("Введите название задачи:")
    await state.set_state(TaskState.waiting_for_name)

@dp.message(TaskState.waiting_for_name)
async def task_name(message: types.Message, state: FSMContext):
    await state.update_data(name=message.text)
    await message.answer("Введите время выполнения задачи:")
    await state.set_state(TaskState.waiting_for_time)

@dp.message(TaskState.waiting_for_time)
async def task_time(message: types.Message, state: FSMContext):
    data = await state.get_data()
    tasks.append({
        "name": data["name"],
        "time": message.text
    })
    await message.answer("Задача добавлена! 💪 Это твой шанс закрыть сделку!")
    await state.clear()


In [ ]:
@dp.message(lambda message: message.text == "Просмотреть задачи")
async def show_tasks(message: types.Message):
    if not tasks:
        await message.answer("Список задач пуст.")
        return

    for i, task in enumerate(tasks):
        keyboard = InlineKeyboardMarkup(inline_keyboard=[
            [InlineKeyboardButton(text="Удалить задачу", callback_data=f"delete_{i}")]
        ])
        await message.answer(f"{task['name']} - {task['time']}", reply_markup=keyboard)

@dp.callback_query(lambda c: c.data.startswith("delete_"))
async def delete_task(callback: types.CallbackQuery):
    index = int(callback.data.split("_")[1])
    tasks.pop(index)
    await callback.message.answer("Задача удалена.")
    await callback.answer()


In [ ]:
@dp.message(lambda message: message.text == "Добавить сделку")
async def add_deal(message: types.Message, state: FSMContext):
    await message.answer("Введите название сделки:")
    await state.set_state(DealState.waiting_for_name)

@dp.message(DealState.waiting_for_name)
async def deal_name(message: types.Message, state: FSMContext):
    await state.update_data(name=message.text)
    await message.answer("Введите сумму сделки:")
    await state.set_state(DealState.waiting_for_amount)

@dp.message(DealState.waiting_for_amount)
async def deal_amount(message: types.Message, state: FSMContext):
    await state.update_data(amount=message.text)
    await message.answer("Введите статус сделки:")
    await state.set_state(DealState.waiting_for_status)

@dp.message(DealState.waiting_for_status)
async def deal_status(message: types.Message, state: FSMContext):
    data = await state.get_data()
    deals.append({
        "name": data["name"],
        "amount": data["amount"],
        "status": message.text
    })
    await message.answer("Сделка добавлена ✅")
    await state.clear()


In [ ]:
@dp.message(lambda message: message.text == "Просмотреть сделки")
async def show_deals(message: types.Message):
    if not deals:
        await message.answer("Список сделок пуст.")
        return

    for i, deal in enumerate(deals):
        keyboard = InlineKeyboardMarkup(inline_keyboard=[
            [InlineKeyboardButton(text="Изменить статус сделки", callback_data=f"status_{i}")]
        ])
        await message.answer(
            f"{deal['name']} | {deal['amount']} | {deal['status']}",
            reply_markup=keyboard
        )

@dp.callback_query(lambda c: c.data.startswith("status_"))
async def change_status(callback: types.CallbackQuery, state: FSMContext):
    index = int(callback.data.split("_")[1])
    await state.update_data(deal_index=index)
    await callback.message.answer("Введите новый статус:")
    await state.set_state(DealStatusState.waiting_for_new_status)
    await callback.answer()

@dp.message(DealStatusState.waiting_for_new_status)
async def update_status(message: types.Message, state: FSMContext):
    data = await state.get_data()
    deals[data["deal_index"]]["status"] = message.text
    await message.answer("Статус обновлен ✅")
    await state.clear()


In [ ]:
@dp.message(lambda message: message.text == "Получить мотивацию")
async def motivation(message: types.Message):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Ты мотивационный коуч для менеджера по продажам."},
            {"role": "user", "content": "Дай короткую мотивационную фразу на день."}
        ]
    )

    await message.answer(response.choices[0].message.content)


In [ ]:
async def main():
    await dp.start_polling(bot)

await main()
